In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

from collections import defaultdict

from model.Base import Base
from model.Device import Device
from model.Flow import Flow
from model.User import User
from model.user_devices import user_devices

from sqlalchemy import create_engine, text, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.pool import NullPool

from IPython.display import display

DB='postgresql+psycopg2:///ucnstudy'

engine = create_engine(DB, echo=False, poolclass=NullPool)
Base.metadata.bind = engine
Session = sessionmaker(bind=engine)

In [3]:
ses = Session()
users = ses.query(User)

devtfc_beg_userdata = defaultdict(list)
devtfc_end_userdata = defaultdict(list)
for user in users:
    sql_user_devices = text('select * from user, user_devices where user_devices.user_id =:user').bindparams(user = user.id)
    user_devices = ses.execute(sql_user_devices)

    #will get the starting time and ending times considering all devices
    print ("user: " + str(user.id) + ' ' + user.username + "=======================")
    quantity_dev = 0
    info_week_beg = {}
    info_week_end ={}
    for dev in user_devices:
        sql_beg_day = text('SELECT distinct devid, flows.startts \
        FROM flows join \
        (SELECT DATE(startts) as date_entered, MIN(startts) as min_time \
        FROM flows \
        WHERE devid = :d_id and extract (hour from startts) > 3 \
        GROUP BY date(startts)) AS grp ON grp.min_time = flows.startts order by flows.startts;').bindparams(d_id = dev.device_id)
        result_beg_day = ses.execute(sql_beg_day)
        sql_end_day = text('SELECT distinct devid, flows.endts \
        FROM flows join \
        (SELECT DATE(endts) as date_entered, MAX(endts) as max_time \
        FROM flows \
        WHERE devid = :d_id \
        GROUP BY date(endts)) AS grp ON grp.max_time = flows.endts order by flows.endts;').bindparams(d_id = dev.device_id)
        result_end_day = ses.execute(sql_end_day)
        sql_beg_day_nolimit = text('SELECT distinct devid, flows.startts \
        FROM flows join \
        (SELECT DATE(startts) as date_entered, MIN(startts) as min_time \
        FROM flows \
        WHERE devid = :d_id \
        GROUP BY date(startts)) AS grp ON grp.min_time = flows.startts order by flows.startts;').bindparams(d_id = dev.device_id)
        result_beg_day_nolimit = ses.execute(sql_beg_day_nolimit)

        devices_result = ses.query(Device).order_by(Device.id)
        devices_platform = {}
        for item in devices_result:
            devices_platform[item.id] = item.platform

        #organize data
        info_end = defaultdict(list)
        for row in result_end_day:
            info_end['devid'].append(row[0])
            info_end['end'].append(row[1])
            
        info_beg = defaultdict(list)
        for row in result_beg_day:
            info_beg['devid'].append(row[0])
            info_beg['start'].append(row[1])

        #add days that only have value before 3 am
        for row in result_beg_day_nolimit:
            timst = row[1]
            in_list = False
            for dt in info_beg['start']:
                if dt.date() == timst.date():
                    in_list = True
            if in_list == False:
                #insert in the correct position
                cont = 0
                for dat in info_beg['start']:
                    if timst.date() > dat.date():
                        cont = cont + 1
                            
                info_beg['start'].insert(cont, timst)
                info_beg['devid'].insert(cont, row[0])

        df_beg = pd.DataFrame(info_beg)
        display(df_beg)
        df_end = pd.DataFrame(info_end)
        display(df_end)
            
            
        #creates table per day for beginning
        info_week = defaultdict(list)
        if (info_beg['start']):
            for timst in info_beg['start']:
                day = timst
                weekday = day.strftime('%A')
                info_week[weekday].append(day)
         
        print('Device platform: ' + devices_platform[dev.device_id])
        days_str = {'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday', 'Sunday'}
        for name in days_str:
            df_col = defaultdict(list)
            df_col['device'] = str(dev.device_id)
            df_col[name+' start'] = info_week[name]
            df_week = pd.DataFrame(df_col)
            display(df_week)
            
            
        #creates table per day for ending
        info_week = defaultdict(list)
        if (info_end['end']):
            for timst in info_end['end']:
                day = timst
                weekday = day.strftime('%A')
                info_week[weekday].append(day)
             
        print('Device platform: ' + devices_platform[dev.device_id])
        days_str = {'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday', 'Sunday'}
        for name in days_str:
            df_col = defaultdict(list)
            df_col['device'] = str(dev.device_id)
            df_col[name+' end'] = info_week[name] 
            df_week = pd.DataFrame(df_col)
            display(df_week)
            
            

user: 1 norelie=======================


,devid,start
0,6,2015-03-05 21:48:08.111112
1,6,2015-03-06 14:06:25.460244
2,6,2015-03-07 11:27:27.914741
3,6,2015-03-08 00:00:09.913389
4,6,2015-03-09 18:45:06.395510
5,6,2015-03-12 10:05:48.055392
6,6,2015-03-14 13:09:53.791906
7,6,2015-03-15 14:16:15.747360
8,6,2015-03-16 19:08:00.564826
9,6,2015-03-17 19:38:47.777504


,devid,end
0,6,2015-03-05 23:57:41.262526
1,6,2015-03-06 23:16:28.906022
2,6,2015-03-07 23:57:04.582814
3,6,2015-03-08 00:16:09.255980
4,6,2015-03-09 22:35:46.926885
5,6,2015-03-12 23:11:25.820677
6,6,2015-03-14 22:30:12.594612
7,6,2015-03-15 15:39:36.854986
8,6,2015-03-16 23:59:59.925316
9,6,2015-03-17 23:58:21.016543


Device platform: windows-laptop


,Monday start,device
0,2015-03-09 18:45:06.395510,6
1,2015-03-16 19:08:00.564826,6
2,2015-04-06 00:04:51.480475,6


...

,Friday end,device


,Wednesday end,device
0,2015-11-25 23:12:28.898685,45


,Thursday end,device


,Sunday end,device


,Saturday end,device
